In [1]:
import textract
import PyPDF2
import numpy as np
import pandas as pd
from tika import parser
import tabula
import re
import requests
import os

In [2]:
keyid_inter = '9cfbf9ed-b6c0-4ab1-9762-9383fb15a5de'
keyid_col = '1c635859-2b31-41b2-9802-572347452d30'
keyid_med = '83de8977-c9be-4663-ab6a-0fb6b70641e2'

In [3]:
filename = 'words-of-champions1'

In [4]:
pdf = '/Users/hema/Downloads/SpellBee/words-of-champions1.pdf'

tika /textract 

In [5]:
txt = parser.from_file(pdf)

In [6]:
text = txt['content'].split('\n\n')
all_text = ''.join(text)

In [7]:
all_text

'\nMicrosoft Word - 2008 Spelling Bee Booklet.doc\nWords Of \nChampionsTheSpellingBeeChamp.wordpress.com \nWebsite by Cole Shafer-Ray The Spelling Bee Champ 3THIRD GRADE SPELLING LIST - (H) = HOMONYM 1. coin\n2. oil\n3. voice\n4. join\n5. noise\n6. poison\n7. boil\n8. point\n9. enjoy\n10. toys\n11. joy\n12. royal\n13. radio\n14. house\n15. loud\n16. mouth\n17. proud\n18. around\n19. about\n20. cows\n21. towel\n22. town\n23. how\n24. power\n25. found\n26. race\n27. certain\n28. bicycle\n29. price\n30. city\n31. pencil\n32. mustard\n33. scare\n34. second\n35. circus\n36. slice\n37. dancer\n38. recycle\n39. knock\n40. knife\n41. knee\n42. never43. remember\n44. rowboat\n45. wrist\n46. wrong\n47. wrote (H)\n48. neighbor\n49. wreck\n50. giant\n51. gym\n52. journey\n53. jam\n54. jacket\n55. next\n56. fixed\n57. except\n58. quart\n59. squirrel\n60. giraffe\n61. quarterback\n62. sleeping\n63. building\n64. camping\n65. crying\n66. eating\n67. feeding\n68. growing\n69. helping\n70. hunting\n71.

In [8]:
all_text = all_text.split('\n')

In [13]:
all_text[3] = 'The Spelling Bee Champ 3THIRD GRADE SPELLING LIST - (H) = HOMONYM'
all_text[4] = '1. coin'

In [15]:
all_text.index('1. coin')


4

In [16]:
df = pd.DataFrame(all_text[3:], columns=['Word'])

In [17]:
print(len(all_text))
print(len(df))

5832
5829


In [41]:
df1 = df.applymap(lambda x: x.replace('\n',''))
df1 = df1.applymap(lambda x: x.replace('TheSpellingBeeChamp.wordpress.com',''))
df1.dropna(inplace=True)
df1 = df1.applymap(lambda x: x.strip())
len(df1
   )

5829

In [42]:
df1 = df1.applymap(lambda x : re.sub(re.compile('\d{1,4}\.'),'',x))

In [43]:
df1 = df1[(df1['Word'].str.lower()!='paideia 2004-2006')&(df1['Word']!='The Spelling Champ')&
          (df1['Word']!='TheSpellingChamp.comWebsite by Cole Shafer-Ray')&(df1['Word']!='Paideia 2004-2006 Words')&
          (df1['Word']!='This word list is a supplement to the CWL(2004) and Spell It! (2007-2012)')&
         (df1['Word']!='Paideia 2004')&(df1['Word']!='TheSpellingChamp.com')]
         
#(df1['Word']!='
len(df1)

5829

In [44]:
df1

,Word
0,The Spelling Bee Champ 3THIRD GRADE SPELLING L...
1,coin
2,oil
3,voice
4,join
...,...
5824,flue flew
5825,gambol gamble
5826,sachet sashay
5827,seine sane


In [45]:
df1 = df1.applymap(lambda x: x.replace('TheSpellingBeeChamp.wordpress.com',''))

In [46]:
df1['Meaning'] = ''

In [47]:
df1

,Word,Meaning
0,The Spelling Bee Champ 3THIRD GRADE SPELLING L...,
1,coin,
2,oil,
3,voice,
4,join,
...,...,...
5824,flue flew,
5825,gambol gamble,
5826,sachet sashay,
5827,seine sane,


In [48]:
words_without_meaning = df1[df1['Meaning']=='']['Word']
respE = [requests.get(f'https://www.dictionaryapi.com/api/v3/references/collegiate/json/{word}?key={keyid_col}') for word in words_without_meaning]


In [49]:
respE[0].json()

[]

In [50]:
words = df1

In [51]:
#def get_word_meaning_mw( words, respE):

all_rows = []
missing_words = []
words_without_meaning = words[words['Meaning']=='']['Word']
# respE = [requests.get(f'https://www.dictionaryapi.com/api/v3/references/collegiate/json/{word}?key={keyid_col}') for word in words_without_meaning]

all_rows = []
meta_dict = {}
meta_cnt = 0
set_miss_word_dict = {}
#filename = 'L1A'
#filepath = file #'/Users/hema/Downloads'
print(all_rows)
aud_cnt = 1
for idx, res in enumerate(respE):   
    ids = ''
    stems = ''
    fls = ''
    hwis = ''
    shdef = ''
    ets = ''
    prss = ''
    audios = ''
    audio = []
    audio_filename = ''
    try:
        data = (res.json())
        #print(data)
        print('=============================================')
        if "'meta'" not in str(data):
              missing_words.append(words.iloc[idx,:].values[0])
        else :
            for meta in data:
                    aud_cnt += 1
                    for key, item in meta.items():
                        meta_dict[key] = item 
                        for key, item in meta_dict.items():
                            if key=='meta':
                                for k,i in item.items():
                                    if k=='id':
                                        ids= i
                                    if k=='stems':
                                        stems = i

                            if key=='fl':
                                fls = item

                            if key=='hwi':

                                for k,i in item.items():
                                    if k=='prs':                            
                                        prs_keys = (i[0].items())
                                        for key,pro in prs_keys:
                                            if key=='mw':
                                                prss=pro
                                            if key=='sound':
                                                audios=pro['audio']
                                                aud = pro['audio']
                                                if re.match('^gg*', aud):
                                                    fol = 'gg'
                                                elif re.match('^[a-zA-Z]', aud):
                                                    fol = aud[0]
                                                else:
                                                    fol = 'number'




                            if key=='shortdef':
                                if len(item) != 0:
                                    shdef = item[0]


                            if key=='et':
                                if len(item) != 0:
                                    ets = item[0][1]

    #                 row = (ids, stems, fls, prss, audio_link, shdef, ets, audio_filepath)
                    #row = (ids, prss, ets, shdef)
                    row = (ids, shdef)
                    #print(row)
                    all_rows.append(row)
            #print(all_rows)
    except Exception as e:
        print(e)
        continue

df_all = pd.DataFrame(all_rows, columns=['Word', 'Short Definition']) 

#add the meaning to the word df
#for w in words_without_meaning:
    #print(df_all.loc[df_all['Word']==w,'Short Definition'].values[0])
 #   words.loc[words['Word']==w,'Meaning'] = df_all.loc[df_all['Word']==w,'Short Definition'].values
#return words, df_all

#write the meaning to the source excel/csv file

df_all









[]
Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


...

Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)


Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (

,Word,Short Definition
0,coin:1,"corner, cornerstone, quoin"
1,coin:2,to make (a coin) especially by stamping : mint
2,coin:3,of or relating to coins
3,coin of the realm,the legal money of a country
4,coin-op,a self-service laundry where the machines are ...
...,...,...
17614,Simon,peter —called also Simon Peter
17615,zephyr,a breeze from the west
17616,zinnia,any of a genus (Zinnia) of tropical American c...
17617,zodiac,an imaginary band in the heavens centered on t...


In [52]:
df_all

,Word,Short Definition
0,coin:1,"corner, cornerstone, quoin"
1,coin:2,to make (a coin) especially by stamping : mint
2,coin:3,of or relating to coins
3,coin of the realm,the legal money of a country
4,coin-op,a self-service laundry where the machines are ...
...,...,...
17614,Simon,peter —called also Simon Peter
17615,zephyr,a breeze from the west
17616,zinnia,any of a genus (Zinnia) of tropical American c...
17617,zodiac,an imaginary band in the heavens centered on t...


In [53]:
df_all.fillna('Meaning', inplace=True)

In [54]:
#words_without_meaning
for w in words_without_meaning:
    print(w)
print(df_all[['Word','Short Definition']].values)

The Spelling Bee Champ 3THIRD GRADE SPELLING LIST - (H) = HOMONYM
 coin
 oil
 voice
 join
 noise
 poison
 boil
 point
 enjoy
 toys
 joy
 royal
 radio
 house
 loud
 mouth
 proud
 around
 about
 cows
 towel
 town
 how
 power
 found
 race
 certain
 bicycle
 price
 city
 pencil
 mustard
 scare
 second
 circus
 slice
 dancer
 recycle
 knock
 knife
 knee
 never remember
 rowboat
 wrist
 wrong
 wrote (H)
 neighbor
 wreck
 giant
 gym
 journey
 jam
 jacket
 next
 fixed
 except
 quart
 squirrel
 giraffe
 quarterback
 sleeping
 building
 camping
 crying
 eating
 feeding
 growing
 helping
 hunting
 studying
 carrying
 interesting
 evening
 skiing
 letting
 tapping
 dropping
 forgetting
 rubbing
 hopping
 hitting
 popping
 running ripping
 tripping
 inning
 biting
 closing
 exciting
 inviting
 shining
 skating
 taking
 deciding
 leaving
 tasting
 shaking
 losing
 squeezing
 scribbling
 pitcher
 catcher
 zipper
 driver
 skater
 laughed
 climbed
 wanted
 spotted
 slipped
 chased
 saved
 skier
 discov

 preacher
 honestly
 overt
 irate
 copies
 ethics
 potent
 unsanitary
 humility
 pardon
 civilize
 cancel
 extinct
 discouraged
 harass
 badge  enclosing
 fiasco
 challenge
 extortion
 sanitary
 flagrant
 youth
 leisure
 cringe
 questionnaire
 sophomore
 delightful
 parallel
 shipped
 qualifications
 justice
 skeptical
 profit (H)
 patient
 crypt
 chrysanthemum
 horrify
 jealousy
 exhibit
 scheme
 sheriff
 intelligent
 spotless
 psychology
 oblige
 innocence
 spite
 various
 obstacle
 digest
 superintendent
 heavily
 ordinary
 merrily
 frequently
 supervisor
 manage  possible
 rubbish
 suspicious
 stunned
 gradual
 vigorously
 neglect
 tongue
 dreadful
 imitative
 glamorous
 tragedy
 impressive
 thorough
 transferred
 political
 pledge
 multiple
 legibly
 secretary
 attractive
 incredible
 persuade
 artificial
 negotiate
 victory
 seize (H)
 miscellaneous
 traitor
 privilege
 beneficial
 mortgage
 serial (H)
 smudge
 obituary
 partial
 starred
 obviously
 souvenir
 absolutely
 optimist

 memorial
 menial
 merchant  mercy
 merge
 merry (H)
 message
 methodical
 midriff
 mighty
 nameplate
 napkin
 narcotic
 nervy
 nettle
 neutral
 nibs
 nicety
 nightmare
 nobility
 noisy
 nomadic
 nominee
 nonsense
 oasis
 oatmeal
 objection
 oblong
 obvious
 occasional
 occupancy
 occurrence
 oceanic
 opal
 operation
 opine
 oppose
 opt
 oration
 ordeal
 ordinary
 origin
 oriole
 orphan  package
 palace
 panda
 papacy
 paradox
 parallel
 parlay
 parley
 parsley
 partial
 particle
 passageway
 passion
 pasteboard
 pastor
 pasture
 patent
 patient
 perception
 perfect
 perforate
 perform
 period
 perish (H)
 perky
 permanent
 permeate
 permissible
 personable
 perverse
 photograph
 piano
 pickle
 piecemeal
 pier (H)
 piety
 pigeonhole
 pigment
 pigskin
 piker
 pilfer  24WORDS OF CHAMPIONS - FIRST ROUND WORDS
(H) = HOMONYM
 pilgrim
 pillow
 pincers
 pineapple
 pinking
 pinto
 pinwheel
 pioneer
 piping
 piston
 pittance
 placement
 plantation
 platitude
 platoon
 playgoer
 pleasantry
 pled

 blatancy
 blazon
 bleary
 blintzes
 blitheful
 bloat
 blooper
 bludgeon
 caboodle
 cacophony
 cadaver
 calamitous
 calculus
 calisthenics
 calligrapher
 callow
 camelback
 camouflage
 campanile
 camphor
 camporee
 camwood
 candelabrum
 cannibalism  cannonade
 canorous
 cantankerous
 capacitance
 capillary
 capsize
 carafe
 carnivorous
 carotid
 carrion
 cartilage
 casserole
 casuistry
 catacomb
 catalytic
 catarrh
 catastasis
 catatonic
 catechism
 cattery
 cavalcade
 centurion
 ceremonious
 chancellor
 chandelier
 chapeau
 chateaubriand
 chattel
 chautauqua
 chauvinistic
 chenille
 cherubic
 cheviot
 chihuahua
 chimerical
 chinchilla
 chiropodist
 choleric
 choriamb
 chrysalis
 chrysanthemum  churlish
 chymous
 cicada
 circumlocution
 cirrhosis
 clairvoyance
 classicist
 clientele
 cloche
 coadjutor
 coalition
 cobalt
 coccyx
 cockatoo
 cockleshell
 cognosce
 colectomy
 collaborator
 collateral
 collimate
 colloquialism
 colonnade
 colophon
 comestible
 commensurate
 commiseration
 c

In [55]:
#df1.reset_index(inplace=True)
df_all.loc[4:6]

,Word,Short Definition
4,coin-op,a self-service laundry where the machines are ...
5,fleur de coin,being in the preserved mint condition
6,initial coin offering,the first sale of a cryptocurrency to the publ...


In [56]:
len(df_all)

17619

In [57]:
'''for w in words_without_meaning:
    #print(df_all.loc[df_all['Word']==w,'Short Definition'].values[0])
    words.loc[words['Word']==w,'Meaning'] = df_all.loc[df_all['Word']==w,'Short Definition'].values[0]'''

"for w in words_without_meaning:\n    #print(df_all.loc[df_all['Word']==w,'Short Definition'].values[0])\n    words.loc[words['Word']==w,'Meaning'] = df_all.loc[df_all['Word']==w,'Short Definition'].values[0]"

In [58]:
#df_all['Short Definition'].rename('Meaning', inplace=True)

In [59]:
#words = get_word_meaning_mw(df1, respE)
df_all.rename(columns={'Short Definition':'Meaning'}, inplace=True)

In [60]:
df_all

,Word,Meaning
0,coin:1,"corner, cornerstone, quoin"
1,coin:2,to make (a coin) especially by stamping : mint
2,coin:3,of or relating to coins
3,coin of the realm,the legal money of a country
4,coin-op,a self-service laundry where the machines are ...
...,...,...
17614,Simon,peter —called also Simon Peter
17615,zephyr,a breeze from the west
17616,zinnia,any of a genus (Zinnia) of tropical American c...
17617,zodiac,an imaginary band in the heavens centered on t...


In [61]:
df_merged = pd.merge(left=df1['Word'], right=df_all, how='outer', on='Word')

In [62]:
len(df_merged)
df_merged

,Word,Meaning
0,The Spelling Bee Champ 3THIRD GRADE SPELLING L...,NaN
1,coin,NaN
2,oil,NaN
3,voice,NaN
4,join,NaN
...,...,...
23443,Simon,peter —called also Simon Peter
23444,zephyr,a breeze from the west
23445,zinnia,any of a genus (Zinnia) of tropical American c...
23446,zodiac,an imaginary band in the heavens centered on t...


In [63]:
writer = pd.ExcelWriter('words-of-champions1.xlsx', engine='xlsxwriter')
for i in range(0,23500,500):
    df_merged.iloc[i:i+500,:].to_excel(writer, sheet_name=str(i+1)+'-'+ str(i+500), index=False)
writer.save()

In [65]:
df_merged[df_merged['Word'].str.contains('breaking')]

,Word,Meaning
5930,breaking point,the point at which a person gives way under st...
